In [33]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [35]:
pip install selenium==4.2.0

Note: you may need to restart the kernel to use updated packages.


In [36]:
pip install --user linkedin_scraper

In [37]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from linkedin_scraper import Person, actions
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from linkedin_scraper.objects import Experience, Education, Scraper, Interest, Accomplishment, Contact
import os
from linkedin_scraper import selectors
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import random
import pickle
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import time



__TOP_CARD = "pv-top-card"
__WAIT_FOR_ELEMENT_TIMEOUT = 5


def scrape(scraper, close_on_complete=True):
        if scraper.is_signed_in():
            scrape_logged_in(scraper, close_on_complete=close_on_complete)
            return 0
        else:
            return -1

def scrape_logged_in(self, close_on_complete=True):
        driver = self.driver
        duration = None

        root = WebDriverWait(driver, __WAIT_FOR_ELEMENT_TIMEOUT).until(
            EC.presence_of_element_located(
                (
                    By.CLASS_NAME,
                    __TOP_CARD,
                )
            )
        )
        self.focus()
        self.wait(5)

        # get name and location
        self.get_name_and_location()

        self.open_to_work = self.is_open_to_work()

        # get about
        get_about(self)
        driver.execute_script(
            "window.scrollTo(0, Math.ceil(document.body.scrollHeight/2));"
        )
        driver.execute_script(
            "window.scrollTo(0, Math.ceil(document.body.scrollHeight/1.5));"
        )

        # get experience
        try:
            get_experiences(self)
        except:
            pass

        # get education
        try:
            get_educations(self)
        except:
            pass


        driver.get(self.linkedin_url)
        
        # get skills
        try:
            get_skills(self)
        except:
            self.skills = []
            pass
        
        # get courses
        try:
            get_courses(self)
        except:
            self.courses = []
            pass

        if close_on_complete:
            driver.quit()
            
def get_about(self):
        try:
            about = self.driver.find_element_by_id("about").find_element_by_xpath("..").find_element_by_class_name("display-flex").text
            self.about = about
        except NoSuchElementException:
            pass
        
def get_experiences(self):
        url = os.path.join(self.linkedin_url, "details/experience")
        self.driver.get(url)
        self.focus()
        main = self.wait_for_element_to_load(by=By.ID, name="main")
        self.scroll_to_half()
        self.scroll_to_bottom()
        main_list = self.wait_for_element_to_load(name="pvs-list", base=main)
        for position in main_list.find_elements_by_xpath("li"):
            position = position.find_element_by_class_name("pvs-entity")
            company_logo_elem, position_details = position.find_elements_by_xpath("*")
            # company elem
            company_linkedin_url = company_logo_elem.find_element_by_xpath("*").get_attribute("href")

            # position details
            position_details_list = position_details.find_elements_by_xpath("*")
            position_summary_details = position_details_list[0] if len(position_details_list) > 0 else None
            position_summary_text = position_details_list[1] if len(position_details_list) > 1 else None
            outer_positions = position_summary_details.find_element_by_xpath("*").find_elements_by_xpath("*")
        
            if len(outer_positions) == 4:
                position_title = outer_positions[0].find_element_by_tag_name("span").find_element_by_tag_name("span").text
                company = outer_positions[1].find_element_by_tag_name("span").text
                work_times = outer_positions[2].find_element_by_tag_name("span").text
                location = outer_positions[3].find_element_by_tag_name("span").text
            elif len(outer_positions) == 3:
                if "·" in outer_positions[2].text:
                    position_title = outer_positions[0].find_element_by_tag_name("span").find_element_by_tag_name("span").text
                    company = outer_positions[1].find_element_by_tag_name("span").text
                    work_times = outer_positions[2].find_element_by_tag_name("span").text
                    location = ""
                else:
                    position_title = ""
                    company = outer_positions[0].find_element_by_tag_name("span").find_element_by_tag_name("span").text
                    work_times = outer_positions[1].find_element_by_tag_name("span").text
                    location = outer_positions[2].find_element_by_tag_name("span").text
            elif len(outer_positions) == 2:
                        position_title = ""
                        company = outer_positions[0].find_element_by_tag_name("span").find_element_by_tag_name("span").text
                        work_times = outer_positions[1].find_element_by_tag_name("span").text
                        location = ""
                    
                    
            times = work_times.split("·")[0].strip() if work_times else ""
            duration = work_times.split("·")[1].strip() if len(work_times.split("·")) > 1 else None

            from_date = " ".join(times.split(" ")[:2]) if times else ""
            to_date = " ".join(times.split(" ")[3:]) if times else ""
            

            if position_summary_text and len(position_summary_text.find_element_by_class_name("pvs-list").find_element_by_class_name("pvs-list").find_elements_by_xpath("li")) > 1:
                descriptions = position_summary_text.find_element_by_class_name("pvs-list").find_element_by_class_name("pvs-list").find_elements_by_xpath("li")
                for description in descriptions:
                    res = description.find_element_by_tag_name("a").find_elements_by_xpath("*")
                    position_title_elem = res[0] if len(res) > 0 else None
                    
                    if len(res) == 2:
                        work_times_elem = res[1] 
                        location_elem = None
                    elif len(res) == 3:
                        if "·" in res[2].find_element_by_xpath("*").text:
                            work_times_elem = res[2]
                            location_elem = None
                        else:
                            work_times_elem = res[1] 
                            location_elem = res[2]
                    elif len(res) == 4:
                        work_times_elem = res[2] 
                        location_elem = res[3]
                    else:
                        work_times_elem = None
                        location_elem = None

                    location = location_elem.find_element_by_xpath("*").text if location_elem else None
                    position_title = position_title_elem.find_element_by_xpath("*").find_element_by_tag_name("*").text if position_title_elem else ""
                    work_times = work_times_elem.find_element_by_xpath("*").text if work_times_elem else ""
                    times = work_times.split("·")[0].strip() if work_times else ""
                    duration = work_times.split("·")[1].strip() if len(work_times.split("·")) > 1 else None
                    from_date = " ".join(times.split(" ")[:2]) if times else ""
                    to_date = " ".join(times.split(" ")[3:]) if times else ""

                    experience = Experience(
                        position_title=position_title,
                        from_date=from_date,
                        to_date=to_date,
                        duration=duration,
                        location=location,
                        description="",
                        institution_name=company,
                        linkedin_url=company_linkedin_url
                    )
                    self.add_experience(experience)
            else:
                description = position_summary_text.find_element_by_tag_name("span").text if position_summary_text else ""

                experience = Experience(
                    position_title=position_title,
                    from_date=from_date,
                    to_date=to_date,
                    duration=duration,
                    location=location,
                    description=description,
                    institution_name=company,
                    linkedin_url=company_linkedin_url
                )
                self.add_experience(experience)

def get_educations(self):
        url = os.path.join(self.linkedin_url, "details/education")
        self.driver.get(url)
        self.focus()
        main = self.wait_for_element_to_load(by=By.ID, name="main")
        self.scroll_to_half()
        self.scroll_to_bottom()
        main_list = self.wait_for_element_to_load(name="pvs-list", base=main)
        for position in main_list.find_elements_by_class_name("pvs-entity"):
            institution_logo_elem, position_details = position.find_elements_by_xpath("*")

            # company elem
            institution_linkedin_url = institution_logo_elem.find_element_by_xpath("*").get_attribute("href")

            # position details
            position_details_list = position_details.find_elements_by_xpath("*")
            position_summary_details = position_details_list[0] if len(position_details_list) > 0 else None
            position_summary_text = position_details_list[1] if len(position_details_list) > 1 else None
            outer_positions = position_summary_details.find_element_by_xpath("*").find_elements_by_xpath("*")
            
            if len(outer_positions) == 1:
                institution_name = outer_positions[0].find_element_by_tag_name("span").find_element_by_tag_name("span").text
                degree = None
                times = None
            elif len(outer_positions) == 2:
                institution_name = outer_positions[0].find_element_by_tag_name("span").find_element_by_tag_name("span").text
                if " - " in outer_positions[1].find_element_by_tag_name("span").text and len(outer_positions[1].find_element_by_tag_name("span").text)==5 and " ".join(times.split(" ")[:1]).isnumeric() and " ".join(times.split(" ")[2:]).isnumeric():
                    degree = None
                    times = outer_positions[1].find_element_by_tag_name("span").text
                else:
                    degree = outer_positions[1].find_element_by_tag_name("span").text
                    times = None
            elif len(outer_positions) == 3:
                institution_name = outer_positions[0].find_element_by_tag_name("span").find_element_by_tag_name("span").text
                degree = outer_positions[1].find_element_by_tag_name("span").text
                times = outer_positions[2].find_element_by_tag_name("span").text

            if times is not None:
                from_date = " ".join(times.split(" ")[:1])
                to_date = " ".join(times.split(" ")[2:])
            else:
                from_date = " "
                to_date = " "

            description = position_summary_text.find_element_by_tag_name("span").text if position_summary_text else ""

            education = Education(
                from_date=from_date,
                to_date=to_date,
                description=description,
                degree=degree,
                institution_name=institution_name,
                linkedin_url=institution_linkedin_url
            )
            self.add_education(education)
            

def get_skills(self):
        url = os.path.join(self.linkedin_url, "details/skills")
        self.driver.get(url)
        self.focus()
        main = self.wait_for_element_to_load(by=By.ID, name="main")
        self.scroll_to_half()
        self.scroll_to_bottom()
        main_list = self.wait_for_element_to_load(name="pvs-list", base=main)
        self.skills = []
        for skill in main_list.find_elements_by_class_name("pvs-entity"):
            self.skills.append(skill.find_element_by_tag_name("span").find_element_by_tag_name("span").text)
            
def get_courses(self):
        url = os.path.join(self.linkedin_url, "details/courses")
        self.driver.get(url)
        self.focus()
        main = self.wait_for_element_to_load(by=By.ID, name="main")
        self.scroll_to_half()
        self.scroll_to_bottom()
        main_list = self.wait_for_element_to_load(name="pvs-list", base=main)
        self.courses = []
        for course in main_list.find_elements_by_class_name("pvs-entity"):
            self.courses.append(course.find_element_by_tag_name("span").find_element_by_tag_name("span").text)
            
def __repr__(self):
        return "{name}\n\nAbout\n{about}\n\nLocation\n{loc}\n\nExperience\n{exp}\n\nEducation\n{edu}\n\nSkills\n{skills}\n\nCourses\n{courses}".format(
            name=self.name,
            about=self.about,
            loc=self.location,
            exp=self.experiences,
            edu=self.educations,
            skills=self.skills,
            courses=self.courses,
        )

In [35]:
profile_urls = [] 
set_profile = []
profiles = [['Software Developer',0], ['Business Analyst',0], ['Marketing Analyst',0], ['Sales Analyst',0], ['UX Designer',0], ['Product Manager',0], ['Account Manager',0], ['Machine Learning Engineer',0]]
  

def get_urls(profile_urls, set_profile, profiles):
    data_points = len(profile_urls)
    while data_points <= 1000:
        job = random.randrange(len(profiles))
        ctr = profiles[job][1]

        query = 'https://google.com/search?q=site:linkedin.com/in AND ' + profiles[job][0] +'&start='+str(ctr)
        response = requests.get(query)

        delays = [7, 6, 10, 19]
        delay = np.random.choice(delays)
        time.sleep(delay)

        soup = BeautifulSoup(response.text,'html.parser')
        for anchor in soup.find_all('a'):
            url = anchor["href"]

            if '//support.google.com/websearch/answer/86640' in url:
                file = open("linkedIn_urls.pickle","wb")
                pickle.dump(profile_urls, file)
                file.close()

                file = open("profile_data.pickle","wb")
                pickle.dump(profiles, file)
                file.close()

                file = open("profile_set.pickle","wb")
                pickle.dump(set_profile, file)
                file.close()

                return profile_urls

            if 'https://www.linkedin.com/' in url:
                url = url[7:url.find('&')]
                if url not in set_profile:
                    profile_urls.append([url, profiles[job][0]])
                    set_profile.append(url)
                    data_points = data_points + 1
                    print(url)
        profiles[job][1] = ctr+10
    return profile_urls

linkedIn_urls = get_urls(profile_urls, set_profile, profiles)

In [51]:
file = open("linkedIn_urls.pickle","rb")
profile_urls = pickle.load(file) 
file.close()

file = open("profile_set.pickle","rb")
set_profile = pickle.load(file) 
file.close()

file = open("profile_data.pickle","rb")
profiles = pickle.load(file) 
file.close()

In [58]:
driver = webdriver.Chrome(ChromeDriverManager().install())
email = "9971774111"
password = "balaji123"
actions.login(driver, email, password)

In [62]:
file = open("linkedIn_dataset.pickle","rb")
data_matrix = pickle.load(file) 
file.close()

counter = len(data_matrix)
url_profile_list = profile_urls[counter:]

for point in url_profile_list:
    url = point[0]
    profile = point[1]
    person1 = Person(url, driver=driver, scrape = False)
    if scrape(person1, close_on_complete=False) != -1:
        row = [__repr__(person1),[profile]]
        data_matrix.append(row)
        print(profile)
    else:
        row = [url,[profile]]
        data_matrix.append(row)
#         driver.quit()
#         driver = webdriver.Chrome(ChromeDriverManager().install())
#         email = "rishita.chauhan2911@gmail.com"
#         password = "348110076"
#         actions.login(driver, email, password)


In [61]:
file = open("linkedIn_dataset.pickle","wb")
pickle.dump(data_matrix, file)
file.close()

In [73]:
data = pd.DataFrame(data = data_matrix, columns=['LinkedIn Resume', 'Job Profiles'])
remove = data[data['LinkedIn Resume'].str.startswith('https://')].index
data.drop(remove, axis = 0, inplace = True)
data

,LinkedIn Resume,Job Profiles
0,Shyvee Shi\n(She/Her)\n3rd degree connection\n...,[Product Manager]
1,Elisa Bellagamba\n\nAbout\nHigh-impact product...,[Product Manager]
2,Joni (Rafalski) Hoadley (She/Her)\n\nAbout\nI’...,[Product Manager]
3,Shane Connelly\n\nAbout\nI lead the product ma...,[Product Manager]
4,Dana Tom\n(She/Her)\n\nAbout\nI’m a product ma...,[Product Manager]
...,...,...
407,Johanna P.\n\nAbout\nExperienced Bilingual Rep...,[Sales Analyst]
408,Rachel Sedlacek\n\nAbout\nAnalyst with 12 year...,[Sales Analyst]
409,Deirdre Derby\n\nAbout\n[]\n\nLocation\nMinnea...,[Sales Analyst]
410,Ryan Mitchell\n\nAbout\nSr. Sales Analyst for ...,[Sales Analyst]


In [79]:
data[data['LinkedIn Resume'].duplicated()]

,LinkedIn Resume,Job Profiles


In [87]:
r = data['Job Profiles'].str[0]
r.value_counts()

Sales Analyst                68
Machine Learning Engineer    68
Product Manager              62
Marketing Analyst            51
UX Designer                  47
Software Developer           28
Business Analyst             24
Account Manager              24
Name: Job Profiles, dtype: int64

In [74]:
data.to_csv('LinkedIn_dataset.csv', index=False)